<a href="https://colab.research.google.com/github/EAakiyama3104/python_lecture/blob/master/%5BPython%E8%AC%9B%E5%BA%A7%5D%E7%AC%AC3%E5%9B%9ESpotify_API%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 今回学ぶこと
- spotify API を使って、曲の特徴や、ジャンルごとの曲取得を行うやり方
- 統計モデルを使って、spotify ストリーミング再生数の予測を行うやり方

## Spotify API とは

* Spotify 上の音楽データを提供しているWeb API
* Spotify上のアーティスト情報や曲の情報などを取得できる

詳細: https://developer.spotify.com/documentation/web-api/reference/

Spotify API の特徴

![代替テキスト](https://cdn-ssl-devio-img.classmethod.jp/wp-content/uploads/2017/12/spotify-api-overview.png)

* Spotify URIやSpotify IDなど、Spotify特有のパラメータ情報をリクエスト、レスポンス時に扱うことがあります。
* レート制限に引っかかるとステータスコード429が返却される。これが返ってきた場合は、Retry-Afterヘッダーの値を確認し、そこに記載されている秒の間はAPIをコールすることができない。

* レスポンスデータは、JSON Object形式で取得します。

* タイムスタンプはUTC zero offset形式（YYYY-MM-DDTHH:MM:SSZ）

* 一部のAPIではページネーションをサポート（offset、limitパラメータで指定）

* 多くのAPIでは、クライアント側でレスポンスデータのキャッシュを行うためのヘッダー情報が付与される。

* レスポンスステータスコードは、RFC 2616 と RFC 6585に則って2xx、3xx、4xx、5xxを返す。

* レスポンスデータのエラー表現には、下記2種類のフォーマットを使用する。

  * Authentication Error Object
  * Regular Error Object
*APIをコールするにはOAuthアクセストークンが必要。

認証方法の種類

![代替テキスト](https://cdn-ssl-devio-img.classmethod.jp/wp-content/uploads/2017/12/Obtaining-Authorization.png)

* App Authorization
  * システム（Spotify）が、クライアントアプリケーションに、Spotify Platform (APIs, SDKs and Widgets)へアクセスすることを許可します。
* User Authorization
  * エンドユーザー（Spotifyのエンドユーザー）が、クライアントアプリケーションに、エンドユーザーが所有するデータへアクセスすることを許可します。

アプリが認証の許可を得るための3つの方法


* Refreshable user authorization: Authorization Code
  * このフローでは、エンドユーザーはクライアントアプリケーションがリソースへアクセスすることを1度だけ許可します。そのため長期間稼働させるアプリケーションに適しています。 アプリケーションにはリフレッシュ可能なアクセストークンを提供されます。ちなみにトークン交換には秘密鍵の送信が含まれるため、ブラウザやモバイルアプリなどのクライアントからではなく、バックエンドサービスなどの安全な場所でこれを実行します。
* Temporary user authorization: Implicit Grant
  * このフローは、JavaScriptを使用するため、リソース所有者のブラウザで実行されるクライアントアプリケーション向けです。サーバー側のコードを使用する必要はありません。リクエストのレート制限は改善されていますが、リフレッシュトークンは提供されていません。このフローはRFC-6749で定義されています。
* Refreshable app authorization: Client Credentials Flow
  * このフローは、サーバー間認証で使用されます。エンドユーザー情報にアクセスしないエンドポイントのみにアクセスが可能です。
  * このセミナーではこの認証方法を使用

## アカウントを作る

1. https://www.spotify.com/ にアクセスしてアカウントを作る。
2. https://developer.spotify.com/my-applications に行ってログイン
3. Create an Application に Application Name と Description を入れる
4. Are you developing a commercial integration? と聞かれたら、この講座では NO を選択
5. チェックボックスをオンにして Submit
6. Client ID と Client Secret メモしておく。APIを呼ぶ際に使います。

## Spotipyのインストール

Spotipy とは、Spotify APIを呼ぶためのPythonライブラリ

* HTTPリクエストを送る操作をライブラリ内で行っているため、リクエストを意識せずに使うことができる
* pip install spotipy でインストール

In [0]:
!pip install spotipy

In [0]:
import spotipy

spotipy.Spotify クラスがAPIへ様々なリクエストを送る

https://spotipy.readthedocs.io/en/latest/#module-spotipy.client

In [0]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID = "ここにクライアントIDを入力"
CLIENT_SECRET = "ここにクライアントシークレットを入力"

# 認証情報をセット
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
# 認証情報を元に、APIクライアントを作成
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

プレイリストを取得

公式リファレンス: https://developer.spotify.com/documentation/web-api/reference/playlists/get-list-users-playlists/

Spotipyリファレンス: https://spotipy.readthedocs.io/en/latest/#spotipy.client.Spotify.user_playlist

例: プレイリストのタイトルとURLを取得する

In [0]:
# Spotify 公式のプレイリストから2つ取得
playlists = sp.user_playlists('spotify',limit=2)
# 辞書形式で返却される
type(playlists)

In [0]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify()

results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
  results = sp.next(results)
  albums.extend(results['items'])

for album in albums:
  print(album['name'])

## 必要なライブラリを読み込み

In [0]:
import pandas as pd
import seaborn as sns

ジャンル情報を取得

In [0]:
pd.DataFrame(sp.recommendation_genre_seeds())

特定のジャンルの曲を取得

/search API を用いる

* 公式リファレンス: https://developer.spotify.com/documentation/web-api/reference/search/search/
* spotipy のリファレンス: https://spotipy.readthedocs.io/en/latest/#spotipy.client.Spotify.search

In [0]:
# ジャンルで検索
search_genre = sp.search(q='genre:acoustic', type='track', offset=0, limit=1)
pprint(search_genre)

In [0]:
# tracks > items から検索結果を取得
pprint(search_genre['tracks']['items'])

In [0]:
# 曲名,idを取得

pprint([f"タイトル : {item['name']}, ID: {item['id']}"  for item in search_genre['tracks']['items']])
# フォーマット文字リテラルと言う仕組みによって、文字列内に変数が展開される　https://note.nkmk.me/python-f-strings/

## トラック ID から曲の特徴を取得

/tracks/get-several-audio-features API を用いる

spotipy では audio_features メソッド

* https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/
* https://spotipy.readthedocs.io/en/latest/#spotipy.client.Spotify.audio_features

danceability

danceability(踊りやすさ)は、テンポ、リズムの安定性、ビートの強さ、全体的な規則性などの音楽要素の組み合わせに基づいて、ダンスのための曲であるかを示します。0.0の値は、最も踊りずらいことを、1.0は、最も踊りやすいことを示します。

energy

エネルギーは、0.0から1.0の指標で、強度およびアクティブ度を表します。一般的には、エネルギッシュなトラックは、速く、音が大きく、騒々しい感じがします。例えば、デスメタルは高いエネルギーを持っていますが、バッハのプレリュードは低い値になります。この属性に寄与する知覚的属性は、ダイナミックレンジ、聴覚が感じる音量、音色、開始時点のレート、および一般的なエントロピーを含みます。

loudness

ラウドネスは、曲の全体的な音量をデシベル（dB）で表したものです。値は曲全体で平均化され、他の曲との相対的なラウドネスを比較するのに役立ちます。値の典型的な範囲は-60から0dbです。

speechiness

スピーチは、曲の中で話された単語の存在を検出します。録音（例えば、トークショー、オーディオブック、詩）のように、音声が占める割合が大きくなるほど、値は1.0に近くなります。0.66を超える値は、ほぼ完全に発声された単語で構成されている曲を表します。0.33と0.66の間の値は、ラップ音楽などのセクションまたはレイヤーのいずれかで、音楽とスピーチの両方を含む可能性がある曲を表します。0.33未満の値は、音楽やその他の非音声のような曲を表す可能性がかなり高いことを示します。

acousticness

曲がアコースティックかどうかを示す0.0から1.0の指標です。 1.0は曲がアコースティックであるということが高いということを意味します。

instrumentalness

曲にボーカルがないかどうかを予測します。この指標では、 “オー(Ooh)”とか “アー(aah)”の音は楽器の出した音として扱われます。ラップや話し言葉はボーカルとして扱われます。インストゥルメンタルネスの値が1.0に近いほど、曲にはボーカル・コンテンツが含まれていない可能性が高くなります。 0.5を超える値は、インストゥルメンタルの曲が通常示す値ですが、値が1.0に近づくほど信頼度が高くなります。

liveness

録音中に聴衆が存在したかを検出します。この値が高いほど、曲がライブで実行された可能性が高くなります。値が0.8を超えると、曲がライブである可能性が高くなります。

valence

曲が伝える音楽のポジティブ性を表す0.0から1.0の尺度。この指数の高い値の曲はより陽性（例えば、幸せ、陽気、陶酔）であり、低い指数の曲はより陰性となります（例えば、悲しい、落ち込んだ、怒る）。

tempo

曲の全体的な推定テンポ。1分あたりのビート(BPM)。音楽用語では、テンポは、ある曲のスピードまたはペースであり、平均ビート期間から導出されます。


以下の記事より引用
https://exploratory.io/note/2ac8ae888097/SpotifyJRockRockHard-Rock-7183595689717906

## 考察ポイント
- [こちら](https://qiita.com/kazuya-n/items/fbee07ef778e166cb6dd#%E3%82%AF%E3%83%A9%E3%82%B9%E9%96%93%E3%81%AE%E9%96%A2%E4%BF%82%E3%81%AE%E5%88%86%E6%9E%90)を参考に、例えば人気の曲の傾向を分析してみるのもいいのかもしれない

In [0]:
# 曲の特徴を取得
pprint(sp.audio_features(tracks=[search_genre['tracks']['items'][0]['id']]))



```
[{'acousticness': 0.469,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5vjLSffimiIP26QG5WcN2K',
  'danceability': 0.618,
  'duration_ms': 198853,
  'energy': 0.443,
  'id': '5vjLSffimiIP26QG5WcN2K',
  'instrumentalness': 0,
  'key': 2,
  'liveness': 0.0829,
  'loudness': -9.681,
  'mode': 1,
  'speechiness': 0.0526,
  'tempo': 119.949,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/5vjLSffimiIP26QG5WcN2K',
  'type': 'audio_features',
  'uri': 'spotify:track:5vjLSffimiIP26QG5WcN2K',
  'valence': 0.167}]
```



## ジャンル毎に100曲を取得

In [0]:
# キーを定数に保存
ACOUSTICNESS = 'acousticness'
DANCEABILITY = 'danceability'
INSTRUMENTALNESS = 'instrumentalness'
LIVENESS = 'liveness'
LOUDNESS = 'loudness'
SPEECHINESS = 'speechiness'
TEMPO = 'tempo'
VALENCE = 'valence'

In [0]:
# データ格納用のdictionaryを用意
tracks = {}
tracks['id'] = []
tracks['name'] = []
tracks['genre'] = []
tracks[ACOUSTICNESS] = []
tracks[DANCEABILITY] = []
tracks[INSTRUMENTALNESS] = []
tracks[LIVENESS] = []
tracks[LOUDNESS] = []
tracks[SPEECHINESS] = []
tracks[TEMPO] = []
tracks[VALENCE] = []

In [0]:
# ジャンル毎に1000個のトラックを取得し、tracks に保存
TRACK_COUNT_PER_GENRE = 100 # ジャンル毎の曲数
TRACK_COUNT_PER_SEARCH = 50 # 1回の検索で取得する曲数
SEARCH_COUNT = int(TRACK_COUNT_PER_GENRE / TRACK_COUNT_PER_SEARCH) # 検索を行う回数 - この場合は2回になる
genres_df = pd.DataFrame(sp.recommendation_genre_seeds()) # ジャンルを取得して、データフレームに変換
for genre in genres_df['genres']:
  for search_count in range(SEARCH_COUNT):
    query = 'genre:' + genre # ジャンルで検索するようにクエリを作成
    offset = TRACK_COUNT_PER_SEARCH * search_count # offsetで検索結果をページのようにズラしていく
    search_results = sp.search(q=query, type='track', offset=offset, limit=TRACK_COUNT_PER_SEARCH) # 検索を行う
    # items が空の場合は、そのジャンルの取得を終了
    if not search_results['tracks']['items']: 
      break
    # 取得したトラックのID
    track_ids = []
    # 各トラックに対して id などを track_ids, tracks に保存
    for track in search_results['tracks']['items']:
      track_ids.append(track.get('id'))
      tracks['id'].append(track.get('id'))
      tracks['name'].append(track.get('name'))
      tracks['genre'].append(genre)
    # 曲の特徴を取得
    audio_features = sp.audio_features(tracks=track_ids)
    # 各トラックに対して、曲の特徴を tracks に保存
    for audio_feature in audio_features:
      tracks[ACOUSTICNESS].append(audio_feature.get(ACOUSTICNESS))
      tracks[DANCEABILITY].append(audio_feature.get(DANCEABILITY))
      tracks[INSTRUMENTALNESS].append(audio_feature.get(INSTRUMENTALNESS))
      tracks[LIVENESS].append(audio_feature.get(LIVENESS))
      tracks[LOUDNESS].append(audio_feature.get(LOUDNESS))
      tracks[SPEECHINESS].append(audio_feature.get(SPEECHINESS))
      tracks[TEMPO].append(audio_feature.get(TEMPO))
      tracks[VALENCE].append(audio_feature.get(VALENCE))

In [0]:
for key in tracks:
  print(f'{key}: {len(tracks[key])}')

In [0]:
tracks_df = pd.DataFrame(tracks)

In [0]:
tracks_df

ジャンルの数と曲数が合わない理由

どのトラックにも該当しないジャンルがある

In [0]:
tracks_df['genre'].unique()

In [0]:
genres_df['genres'].unique()

## Spotify データでトレンドを推定する。(時系列分析)

### 時系列分析とは?
時間とともに変化する現象を分析する事
ex) 株価推移の予測, ウィルスの感染者推移など

今回はSARIMAモデル（季節調整済み自己回帰和分移動平均モデル)を用いて、トレンドをストリーミング数を予測する。

## 週間チャートをまとめたcsvを読み込む(前回用いたcsv）
こちらのスクリプトによって、チャートのデータを取得することができる
https://github.com/fbkarsdorp/spotify-chart

In [0]:
#ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [0]:
spotify_streaming = pd.read_csv("drive/My Drive/chart_weekly.csv").iloc[:,1:]

In [0]:
spotify_streaming
# Position :順位, Track Name: 曲名, Artist: アーティスト名,Streams: 再生回数, URL:曲のURL,  Date：取得日にち

## 打上花火のトレンドをみてみる。

In [0]:
UchiageHanabi_trend = spotify_streaming[spotify_streaming["Track Name"] == "打上花火"].sort_values('date')
UchiageHanabi_trend.index = spotify_streaming[spotify_streaming["Track Name"] ==  "打上花火"].sort_values('date').date # index を日付にする。

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.subplots( figsize=(20, 10))
UchiageHanabi_trend.Streams.plot()

## 今回は、2019年6月30日以降の再生数を統計モデルを使って予測する。

In [0]:
#2019年を予測検証データに用いるため分ける
# 訓練用のデータ
train= UchiageHanabi_trend[ 
    (UchiageHanabi_trend.date <= "2019-06-30")].set_index("date")[["Streams"]]
# test（検証用のデータ)
test = UchiageHanabi_trend[UchiageHanabi_trend.date > "2019-06-30"].set_index("date")[["Streams"]]

## 使用する統計モデルについて
### 自己回帰（Auto Regression）モデル
現在のデータは、過去のデータからどれだけ変化したかで表すことができるモデル。

現在のデータ = 過去データ + ランダムの変化量
### 移動平均(Moving Average)モデル

現在のデータを、平均値＋ある時点までのホワイトノイズ（満遍なく出現されるランダムな数字）の合計+それらを調整する定数で表現することができるモデル(移動平均の計算と異なることに注意！）

###それを合わせたのをARMAモデルと呼ぶ

→でもそれだと「非定常性」の時系列に対応できない。。。。
### 定常、非定常とは
自己共分散（相関係数のパーツ）が時点によって定まらず、時間差によってのみ定まること（その時系列は弱定常だと言える）
$$ Corr(y_t,y_{t-k}) = \frac{Cov(y_t,y_t-k)}{\sqrt{var(y_t)}\sqrt{var(y_{t-k})}}= \frac{\gamma_k}{\gamma_0} = \rho_k$$
簡単に言うと、長期的には平均に収束する（回帰する）系列を定常過程と呼ぶ。

グラフにするとこんな感じ（ホワイトノイズの例）

![代替テキスト](https://upload.wikimedia.org/wikipedia/commons/8/8b/White_noise.png)

また逆に非定常の過程では平均には収束しない。

グラフにするとこんな感じ(月ごとの飛行機の乗客数の例）
![代替テキスト](https://logics-of-blue.com/wp-content/uploads/2017/05/python-time-series-1-data.png)
株価などのランダムウォークのようなものは下がり続けることもあるし、上がり続けることもありますので平均に回帰するとは言いにくいです。（ちなみに、純粋なランダムウォークでは、一つ昔のデータと、現在のデータの差である階差の過程が定常過程になる。１階分の階差で定常過程になるもの単位根過程、２階分以上の階差で定常過程になるものを和分過程とよぶ）
参考：https://to-kei.net/time-series-analysis/time_analy_random_walk/#i-6

## 非定常過程（の中でも和分過程）に対しても推定できるようにするのがARIMAモデル

さらに、季節などといった周期性のあるものを捉えられるようにするために必要なのがSARIMAモデル




In [0]:
# statsmodels と言う、統計処理用のライブラリを用いる。
#(ver0.11でないと、不具合もあるため、バージョンを指定する。)
!pip install statsmodels==0.11


## 階差と、コレログラムをプロット

In [0]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
plt.rcParams["font.size"] = 10
fig, ax = plt.subplots(2, 3, figsize=(20, 8))
axes = ax.flatten()
diff_color = "blue"
# 階差を取得
raw_data = train.Streams
train_diff1 = raw_data.diff()
axes[0].plot(raw_data)
axes[0].set_title("raw_data")
fig = sm.graphics.tsa.plot_acf(raw_data, lags=52, ax=axes[1])
fig = sm.graphics.tsa.plot_pacf(raw_data, lags=52, ax=axes[2])

axes[3].plot(train_diff1.dropna(), color=diff_color, alpha=1)
axes[3].set_title("diff1")
fig = sm.graphics.tsa.plot_acf(train_diff1, lags=52, ax=axes[4], color=diff_color)
fig = sm.graphics.tsa.plot_pacf(train_diff1, lags=52, ax=axes[5], color=diff_color)
# 左から、時系列グラフ、自己相関係数。偏自己相関係数（コレログラム）をプロットしたもの
# 下図は、階差をした上でのグラフとなっている。

In [0]:
# ADF検定(上の時系列が単位根過程であるかを検定する)
sm.tsa.stattools.adfuller(raw_data, 
                                   maxlag=None, 
                                   regression='c', 
                                   autolag='AIC', 
                                   store=False, 
                                   regresults=False)[1]


この値が
**0.05の場合、単位根過程がなく、SARIMAが使えないので注意** 

## AIRMAで予測する

In [0]:
from statsmodels.tsa import arima_model
import matplotlib.pyplot as plt

results=arima_model.ARIMA(train.Streams.astype("float64").reset_index(drop=True),order = [5,0,5]).fit()
plt.clf()
# plt.plot(pd.concat([train.Streams, test.Streams]).reset_index(drop=True),c="blue", alpha=0.5)
# plt.plot(results.predict(start=1,end=133),c="red")
# plt.legend(["data","predict"])
predicted=results.predict(start=97,end=132).reset_index(drop=True)
predicted.index =test.Streams.index
plt.subplots( figsize=(50, 10))
plt.plot(test.Streams,c="blue", alpha=0.5)
plt.plot(predicted,c="red")
plt.legend(["data","predict"])

あまりうまく適合してない...。

## sarimaxで予測する。
- ARIMA 過程に季節調整(seasonal arrange 時系列データに季節といった周期性があるもの)を加える

In [0]:
model= sm.tsa.statespace.SARIMAX(train.Streams, order=(1,0,1),
                                 seasonal_order= (1,1, 2, 50),#(1,0,0)
                                 trend="ct",
                                 enforce_stationarity = False,
                                 enforce_invertibility=False)
results = model.fit()
print(results.summary())

## テストデータの予測

In [0]:
predicted=results.predict(start=97,end=132).reset_index(drop=True)
predicted.index =test.Streams.index
plt.subplots( figsize=(50, 10))
plt.plot(test.Streams,c="blue", alpha=0.5)
plt.plot(predicted,c="red")
plt.legend(["data","predict"])

## 全期間の予測

In [0]:
total_term=pd.concat([train.Streams, test.Streams])
plt.plot(total_term.reset_index(drop=True),c="blue", alpha=0.5)
plt.plot(results.predict(start=1,end=len(total_term)).reset_index(drop=True),c="red")
plt.legend(["data","predict"])

In [0]:
results.mle_retvals # converged がTrue になっていること(最尤法が収束していること)を確認

## 任意課題

## sarimaxのパラメータを最適化する
- パラメータは７つある。（ ARIMAX用のパラメータ 3つ + 季節調整用のARIMA パラメータ 3つ　+ 季節周期 1つ）
- そのうち、どのパラメータの組み合わせが最適なのかをAICと言う指標を使う(参考：https://bellcurve.jp/statistics/blog/15754.html　指標が小さいほど、有用である可能性が高い)


In [0]:
import itertools
p = [0,3]
d = [0,1]
q = [0,5]
sp = [0,1]
sd = [0,1]
sq = [0,2]
pdq = list(itertools.product(p, d, q))
# P, D, Q, 季節調整用 SP, SD, SQ　の組み合わせを列挙するリストを作成すると同時に、最後の　s = 52　を決め打ちでつけている。
seasonal_pdq = [[x[0], x[1], x[2], x[3],x[4],x[5], 52] for x in list(itertools.product(p,d,q,#))]
                                                                     sp, sd, sq))]
print(seasonal_pdq,f"配列長は{len(seasonal_pdq)}")

In [0]:
def sarimax_fit(train_data,param):
    try:
        result = sm.tsa.statespace.SARIMAX(train_data,
                                        order=(param[0], param[1], param[2]),
                                   seasonal_order=(param[3], param[4], param[5],param[6]),
                                        enforce_invertibility = False, enforce_stationarity = False,
                                          trend="ct"
                                          ).fit()
        return result
    except Exception as e:
        print(e)
        return 0

In [0]:

import joblib
res_list = joblib.Parallel(n_jobs=-1, verbose=10)([joblib.delayed(sarimax_fit)(train.Streams,param) for param in seasonal_pdq])
 
# # AICが小さくなる順にのパラメータの組み合わせを並べ変え
res_list

### AICを取り出す

In [0]:
best_aic=1000
best_col = -1
for i,res in enumerate(res_list):
    if best_aic > res.aic:
        best_aic = res.aic
        
        print(res.aic)
        best_col = i
## もっとも少ないAIC をもつモデルを代入
best_sarimax= res_list[best_col]
aic_df = pd.DataFrame({"params": seasonal_pdq, "aic": [r.aic if r == 0 else 0 for r in res_list]})
aic_df.sort_values("aic", inplace=True)
aic_df

In [0]:
aic_df.head(10)

In [0]:
import itertools
# p = [4,5]
# d = [0,1,2]
# q = [3,4,5]
sp = range(3)
sd = range(2)
sq = range(3)
# pdq = list(itertools.product(p, d, q))
# trial_num = len(p_list) * len(d_list) * len(q_list) *len(sp_list) * len(sd_list) * len(sq_list)

use_columns = ["p", "d", "q", "AIC"]
step_cnt = 0
train_data = raw_data

info_df = pd.DataFrame(columns=use_columns)
for i,param in enumerate(pdq):
    model = sm.tsa.statespace.SARIMAX(train.Streams, order=param, enforce_invertibility=False,
#                                               trend='t'
                                      
                                     )
    p,d,q = param

    try:
        
        res = model.fit(disp=False)
        info_df = info_df.append(pd.Series([p,d,q,res.aic], index=use_columns),ignore_index=True)
    except:
        pass
    print(f"iter {i} finished... param :{param}, aic{res.aic}")
